<a href="https://colab.research.google.com/github/nikhil-xyz/Natural_Language_Processing/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikhil-xyz/Natural_Language_Processing.git

Cloning into 'Natural_Language_Processing'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 8), reused 18 (delta 4), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Natural_Language_Processing/data/Fake_News_Classifier/Fake_news.csv')

In [4]:
df = df.dropna()

In [5]:
X = df.drop('Label', axis=1)
y = df['Label']

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [8]:
vocab_size = 5000

In [9]:
messages = X.copy()
messages.reset_index(inplace=True)

In [10]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['Body'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [12]:
onehot_rep = [one_hot(words, vocab_size) for words in corpus]

In [13]:
sentence_len = 2000

In [14]:
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sentence_len)

In [15]:
len(embedded_docs)

3988

In [16]:
embedding_vector_features = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sentence_len))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
import numpy as np
X = np.array(embedded_docs)
y = np.array(y)

In [19]:
X.shape

(3988, 2000)

In [20]:
y.shape

(3988,)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [22]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Epoch 1/5
100/100 [==============================] - 684s 7s/step - loss: 0.2835 - accuracy: 0.9078 - val_loss: 0.0959 - val_accuracy: 0.9674
Epoch 2/5
100/100 [==============================] - 630s 6s/step - loss: 0.0440 - accuracy: 0.9865 - val_loss: 0.0621 - val_accuracy: 0.9724
Epoch 3/5
100/100 [==============================] - 589s 6s/step - loss: 0.0123 - accuracy: 0.9975 - val_loss: 0.0595 - val_accuracy: 0.9825
Epoch 4/5
100/100 [==============================] - 604s 6s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0524 - val_accuracy: 0.9850
Epoch 5/5
100/100 [==============================] - 592s 6s/step - loss: 5.7317e-04 - accuracy: 1.0000 - val_loss: 0.0560 - val_accuracy: 0.9850


In [28]:
y_pred = (model.predict(X_test)>0.5).astype(int)

25/25 [==============================] - 40s 2s/step


In [29]:
y_pred[:5]

array([[0],
       [0],
       [1],
       [0],
       [0]])

In [27]:
y_test[:5]

array([0, 0, 1, 0, 0])

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[431,   5],
       [  7, 355]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9849624060150376